# This is part of the supporting information for the paper  
*ParAMS: Parameter Fitting for Atomistic and Molecular Models* (DOI: *123123*)  
The full documentation can be found at https://www.scm.com/doc.trunk/params/index.html

# SCC-DFTB repulsive potential parametrization


* The initial point $x_0$ for this optimization is $A_0 = A_1 = A_2 = A_3 = 1$.
* There are four target quantities: $a$ (wurtzite), $c$ (wurtzite), $B_0$ (wurtzite), and $\Delta E$ = relative energy between the wurtzite and rocksalt polymorphs (per ZnO formula unit).
 

In [1]:
import os, sys
import numpy as np
from os.path    import join as opj
from scm.params import *
from scm.params import __version__ as paramsver
from scm.plams import *

num_processes = 4

INDIR = '../dftbdata'
if not os.path.exists(INDIR):
    os.makedirs(INDIR)
    


print(f"ParAMS Version used: {paramsver}")

ParAMS Version used: 0.5.0



# Step 1: Define the job collection
This adds lattice optimizations of the wurtzite and rocksalt polymorphs of ZnO to the job collection.

For wurtzite, the elastic tensor is calculated. From the output, the bulk modulus can then be extracted.

The job collection is stored in jobcollection.yml.

In [2]:
wurtzite, rocksalt = Molecule(opj(INDIR, 'w.xyz')), Molecule(opj(INDIR, 'rs.xyz')) 
print("### Wurtzite ###")
print(wurtzite)
print("### Rocksalt ###")
print(rocksalt)

w_opt_s = Settings()
w_opt_s.input.ams.Task = 'GeometryOptimization'
w_opt_s.input.ams.GeometryOptimization.OptimizeLattice = 'Yes'
rs_opt_s = w_opt_s.copy()
w_opt_s.input.ams.Properties.ElasticTensor = 'Yes' # to get bulk modulus of wurtzite

jc = JobCollection()
jc.add_entry('wurtzite_lattopt', JCEntry(w_opt_s, wurtzite))
jc.add_entry('rocksalt_lattopt', JCEntry(rs_opt_s, rocksalt))
print("### Job collection ###")
print(jc)
jc.store(opj(INDIR, 'jobcollection.yml'))

### Wurtzite ###
  Atoms: 
    1        Zn      1.645000      0.949741      0.023375 
    2        Zn      0.000000      1.899482      2.678375 
    3         O      1.645000      0.949741      3.295375 
    4         O      0.000000      1.899482      0.640375 
  Lattice:
        3.2900000000     0.0000000000     0.0000000000
       -1.6450000000     2.8492235800     0.0000000000
        0.0000000000     0.0000000000     5.3100000000

### Rocksalt ###
  Atoms: 
    1        Zn      2.170000      2.170000      2.170000 
    2         O      0.000000      0.000000      0.000000 
  Lattice:
        0.0000000000     2.1700000000     2.1700000000
        2.1700000000     0.0000000000     2.1700000000
        2.1700000000     2.1700000000     0.0000000000

### Job collection ###
---
ID: rocksalt_lattopt
ReferenceEngineID: None
AMSInput: |
   Task GeometryOptimization
   geometryoptimization
     OptimizeLattice Yes
   End
   system
     Atoms
                Zn      2.1700000000      2.1700

# Step 2: Define the training set
There are four target quantities
* $a$ wurtzite lattice parameter, 
* $c$ wurtzite lattice parameter, 
* $B_0$ wurtzite bulk modulus, and
* and $\Delta E$ = relative energy between the wurtzite and rocksalt polymorphs (per ZnO formula unit).

DFT-calculated reference values are taken from https://doi.org/10.1021/jp404095x

In [3]:
training_set = DataSet()
training_set.add_entry('bulkmodulus("wurtzite_lattopt")', weight=1, reference=129) # GPa
training_set.add_entry('lattice("wurtzite_lattopt", 0)', weight=1, reference=3.29) # a, angstrom
training_set.add_entry('lattice("wurtzite_lattopt", 2)', weight=1, reference=5.31) # c, angstrom
training_set.add_entry('energy("wurtzite_lattopt")/2.0-energy("rocksalt_lattopt")', weight=1, reference=-0.30/27.211)
print("### Training set ###")
print(training_set)
training_set.store(opj(INDIR, 'trainingset.yml'))

### Training set ###
---
Expression: bulkmodulus("wurtzite_lattopt")
Weight: 1
ReferenceValue: 129
---
Expression: lattice("wurtzite_lattopt", 0)
Weight: 1
ReferenceValue: 3.29
---
Expression: lattice("wurtzite_lattopt", 2)
Weight: 1
ReferenceValue: 5.31
---
Expression: energy("wurtzite_lattopt")/2.0-energy("rocksalt_lattopt")
Weight: 1
ReferenceValue: -0.011024953143949138
...



Set the settings for the parametrized DFTB engine. Here, we set the k-space quality to 'Good', which is important for lattice optimizations.

In [4]:
dftb_s = Settings()
dftb_s.input.dftb.kspace.quality = 'Good'

Create a "parameter interface" to the DFTB repulsive potential. 

Repulsive potentials are stored as splines towards the end of Slater-Koster (.skf) files.

Here, we optimize only the Zn-O and O-Zn repulsive potentials (which must be identical).

* Take electronic parameters and unchanged repulsive potentials (e.g. O-O.skf) from AMSHOME/atomicdata/DFTB/DFTB.org/znorg-0-1

* Define an analytical repulsive function. Here, we choose a tapered double exponential of the form $V^{\text{rep}}(r) = f^{\text{cut}}(r)\left[p_0\exp(-p_1r) + p_2\exp(-p_3r)\right]$, where $p_0, p_1, p_2, p_3$ are the parameters to be fitted, and $f^\text{cut}(r)$ is a smoothly decaying cutoff function decaying to 0 at $r = 5.67$ bohr.

* r_range specifies for which distances to write the repulsive potential, and spline parameters, to the new O-Zn.skf and Zn-O.skf files.

* Only optimize parameters for the O-Zn pair. Note: The Zn-O repulsive potential will be identical to the O-Zn one. When specifying active parameters for a DFTBSplineRepulsivePotentialParams, the elements must be ordered alphabetically.

* Define initial values and allowed ranges for the parameter values.

In [5]:
interface = DFTBSplineRepulsivePotentialParams(
    folder=opj(os.environ['AMSHOME'],'atomicdata', 'DFTB', 'DFTB.org', 'znorg-0-1'), 
    repulsive_function=TaperedDoubleExponential(cutoff=5.67), 
    r_range=np.arange(0., 5.87, 0.1), 
    other_settings=dftb_s
)
for p in interface:    
    p.is_active = p.name.startswith('O-Zn:')

print("### Active parameters ###")
interface.active.x = [1.0, 1.1, 1.2, 1.3] # initial values
interface.active.range = [ (0.,10.), (0.,10.), (0.,10.), (0.,10) ]
for p in interface.active:
    print(p)


### Active parameters ###
..................
Name:     O-Zn:p0
Value:    1.0
Range:    (0.0, 10.0)
Active:   True

..................
Name:     O-Zn:p1
Value:    1.1
Range:    (0.0, 10.0)
Active:   True

..................
Name:     O-Zn:p2
Value:    1.2
Range:    (0.0, 10.0)
Active:   True

..................
Name:     O-Zn:p3
Value:    1.3
Range:    (0.0, 10)
Active:   True



# Step 3: Run the optimization
* Specify a Nelder-Mead optimizer from scipy.

In [6]:

optimizer = Scipy(method='Nelder-Mead')

optimization = Optimization(jc, 
                            training_set, 
                            interface, 
                            optimizer, 
                            title="ZnO_repulsive_opt",
                            use_pipe=False, 
                            parallel=ParallelLevels(processes=num_processes), 
                            #plams_workdir_path=os.path.abspath('.'),
                            callbacks=[Logger(printfreq=1,
                                              writefreq_history=1,
                                              writefreq_datafiles=1,
                                              writefreq_bestparams=1
                                             ),
                                      TimePerEval(printfrequency=10)])


optimization.summary()
optimization.optimize()

Optimization() Instance Settings:
Title:                             ZnO_repulsive_opt
Workdir:                           /home/hellstrom/latex/params/params_si.git/trunk/notebooks/ZnO_repulsive_opt
JobCollection size:                2
Interface:                         DFTBSplineRepulsivePotentialParams
Active parameters:                 4
Optimizer:                         Scipy
Parallelism:                       ParallelLevels(optimizations=1, parametervectors=2, jobs=1, processes=4, threads=1)
Verbose:                           True
Callbacks:                         Logger
                                   TimePerEval

Evaluators:
-----------
Name:                              trainingset (_LossEvaluator)
Loss:                              SSE
Evaluation frequency:              1

Data Set entries:                  4
Data Set jobs:                     2
Batch size:                        None

Use PIPE:                          False
---
===
[2020-12-15 17:20:08] Starting paramet

/home/hellstrom/adfhome/bin/python3.6/lib/python3.6/site-packages/scipy/optimize/_minimize.py:517: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


[2020-12-15 17:23:50] Best trainingset loss = 1.138e+02
[2020-12-15 17:23:50] Step 1, trainingset loss = 113.776, first 4 params = 1.00 1.10 1.20 1.30
[2020-12-15 17:25:34] Step 2, trainingset loss = 149.347, first 4 params = 0.80 1.10 1.20 1.30
[2020-12-15 17:28:10] Best trainingset loss = 3.106e+01
[2020-12-15 17:28:10] Step 3, trainingset loss = 31.062, first 4 params = 1.00 0.90 1.20 1.30
[2020-12-15 17:30:04] Step 4, trainingset loss = 126.171, first 4 params = 1.00 1.10 1.01 1.30
[2020-12-15 17:32:09] Step 5, trainingset loss = 47.294, first 4 params = 1.00 1.10 1.20 1.11
[2020-12-15 17:34:43] Step 6, trainingset loss = 34.105, first 4 params = 1.20 1.00 1.10 1.21
[2020-12-15 17:37:20] Step 7, trainingset loss = 39.476, first 4 params = 1.10 0.95 1.34 1.16
[2020-12-15 17:39:55] Step 8, trainingset loss = 122.229, first 4 params = 1.15 0.88 1.22 1.09
[2020-12-15 17:41:44] Step 9, trainingset loss = 60.705, first 4 params = 1.04 1.05 1.21 1.25
[2020-12-15 17:44:11] Step 10, trainin

KeyboardInterrupt: 

# Step 4: Find the results
* ZnO_repulsive_opt/trainingset_history.dat contains the loss function value and parameters for each iteration
* ZnO_repulsive_opt/data/predictions/trainingset contains the individual predictions ($a$, $c$, $B_0$ and $\Delta E$) for each parameter set
* ZnO_repulsive_opt/data/contributions/trainingset contains the fraction of the total loss function value for each item in the training set, for each parameter set

# Step 5: Recalculate the reference data with AMS BAND
Any engine, or combination of different engines in the Amsterdam Modeling Suite, can be used to seamlessly calculate the reference data, if the reference values are not known beforehand.

The `range` attribute allows to define box constraints for every optimizer: